# HouseBrain Model Fine-Tuning on Google Colab

This notebook provides a step-by-step guide to fine-tune the HouseBrain model using Google Colab's free GPU resources. This is the recommended way to train the model, as it avoids local hardware limitations (RAM, GPU, CUDA drivers).


## Step 1: Set Up the Environment

First, we need to connect to a Colab runtime (a T4 GPU is recommended and free). Then, we will clone the project repository from GitHub and install the required Python libraries.


In [ ]:
# Clone the repository (replace with your repository URL)
!git clone https://github.com/your-username/housebrain_v1_1.git
%cd housebrain_v1_1

# Install the necessary libraries
# We install a specific version of torch compatible with Colab's CUDA version
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade transformers peft trl accelerate datasets bitsandbytes sentencepiece jsonschema pydantic


## Step 2: Authenticate with Hugging Face

To download the base model from Hugging Face, you need to provide an access token. You can get a token from your Hugging Face account settings.

When you run the cell below, a login box will appear. Paste your token there.


In [ ]:
from huggingface_hub import login

login()


## Step 3: Generate High-Quality "Silver Standard" Data

Before training, we will leverage the Colab A100 GPU to generate a larger, high-quality dataset. The `generate_silver_standard_data.py` script uses a powerful generate-and-refine loop to create architecturally sound examples automatically. We will generate 100 new examples.

**Note:** This step will take some time as it involves hundreds of LLM calls, but it is crucial for model quality.


In [ ]:
# First, ensure the local Ollama model is being served correctly in the background
# This command starts the server. You may need to run the next cell in a few moments.
!ollama serve &

# Wait a few seconds for the server to initialize
import time
time.sleep(15)

# Pull the model to ensure it's available
!ollama pull deepseek-coder:6.7b-instruct

# Now, run the data generation script
!python scripts/generate_silver_standard_data.py --num-examples 100


## Step 4: Prepare All Datasets for Fine-Tuning

The fine-tuning script requires the `output` field in our JSON examples to be a string. The `prepare_data_for_finetuning.py` script handles this conversion. We will run it on both our original "Gold" dataset and our newly generated "Silver" dataset.


In [ ]:
# Prepare the Gold Standard dataset
!python scripts/prepare_data_for_finetuning.py \
    --input-dir data/training/gold_standard \
    --output-dir data/training/gold_standard_finetune_ready

# Prepare the newly generated Silver Standard dataset
!python scripts/prepare_data_for_finetuning.py \
    --input-dir data/training/silver_standard \
    --output-dir data/training/silver_standard_finetune_ready


## Step 5: Run the Fine-Tuning Script

Now we are ready to fine-tune the model on our combined dataset. For a Colab Pro+ A100 environment, we can use a larger batch size and sequence length to accelerate training and improve performance.

We will point the training script to both the `gold_standard_finetune_ready` and `silver_standard_finetune_ready` directories.


In [ ]:
!python scripts/run_finetuning.py \
    --model_id deepseek-ai/deepseek-coder:6.7b-instruct \
    --dataset_path data/training/gold_standard_finetune_ready data/training/silver_standard_finetune_ready \
    --output_dir models/housebrain-v1.0-silver \
    --epochs 15 \
    --batch_size 4 \
    --learning_rate 0.0002 \
    --use_4bit


## Step 6: (Optional) Download the Trained Model

After training is complete, the new model adapter will be saved in the `models/housebrain-v1.0-silver` directory. You can zip it and download it to your local machine for future use.


In [ ]:
!zip -r housebrain-v1.0-silver-adapter.zip models/housebrain-v1.0-silver

from google.colab import files
files.download('housebrain-v1.0-silver-adapter.zip')


## Step 3: Run the Fine-Tuning Script

Now we are ready to run the fine-tuning script. We will use 4-bit quantization (`--use_4bit`) to ensure the model fits comfortably within the Colab GPU's memory. The script will train the model on our 10 Gold Standard examples for 10 epochs and save the resulting LoRA adapter to the `models/housebrain-v0.1` directory.


In [ ]:
!python scripts/run_finetuning.py \
    --model_id deepseek-ai/deepseek-coder-6.7b-instruct \
    --dataset_path data/training/gold_standard_finetune_ready \
    --output_dir models/housebrain-v0.1 \
    --epochs 10 \
    --batch_size 1 \
    --learning_rate 0.0002 \
    --use_4bit


## Step 4: (Optional) Download the Trained Model

After training is complete, the new model adapter will be saved in the `models/housebrain-v0.1` directory inside the Colab environment. If you want to save it permanently, you can zip it and download it to your local machine.


In [ ]:
!zip -r housebrain-v0.1-adapter.zip models/housebrain-v0.1

from google.colab import files
files.download('housebrain-v0.1-adapter.zip')
